In [1]:
# Import libraries

import warnings
import urllib.request
from pathlib import Path
import os

import torch
import scgpt as scg
import scanpy as sc
import numpy as np
import pandas as pd
import gdown

warnings.filterwarnings("ignore")

/Users/doblado/miniforge3/envs/envTFG/lib/python3.10/site-packages/scgpt/model/model.py:21: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")
/Users/doblado/miniforge3/envs/envTFG/lib/python3.10/site-packages/scgpt/model/multiomic_model.py:19: UserWarning: flash_attn is not installed
  warnings.warn("flash_attn is not installed")


In [3]:
warnings.simplefilter("ignore", ResourceWarning)
warnings.filterwarnings("ignore", category=ImportWarning)

output_dir = "../Experimentacion/save/scGPT_human"
if os.path.exists(output_dir):
    print("model checkpoint already downloaded!!")
else:
    # Use gdown with the recursive flag to download the folder
    # Replace the folder ID with the ID of your folder
    folder_id = "1oWh_-ZRdhtoGQ2Fw24HP41FgLoomVo-y"
    # Download the folder and its contents recursively
    gdown.download_folder(f"https://drive.google.com/drive/folders/{folder_id}", output=output_dir, quiet=False)

model checkpoint already downloaded!!


In [4]:
# This code downloads the data temporarily
uri = "https://datasets.cellxgene.cziscience.com/f50deffa-43ae-4f12-85ed-33e45040a1fa.h5ad"
source_path = "../Experimentacion/data/source.h5ad"
urllib.request.urlretrieve(uri, filename=source_path)
adata = sc.read_h5ad(source_path)

batch_key = "sample"
N_HVG = 3000

"""
    Filtramos el dataset para obtener aquellos genes con una mayor variabilidad y que más
    info nos aportan
"""
sc.pp.highly_variable_genes(adata, n_top_genes=N_HVG, flavor='seurat_v3')
adata_hvg = adata[:, adata.var['highly_variable']]

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [5]:
adata_hvg

View of AnnData object with n_obs × n_vars = 11103 × 3000
    obs: 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'cell_line', 'epcam_positive', 'vim_positive', 'condition_new', 'annotation_res0.34_new2', 'annotation_res0.34_simple2', 'compartment_new2', 'organism_ontology_term_id', 'tissue_ontology_term_id', 'tissue_type', 'assay_ontology_term_id', 'disease_ontology_term_id', 'cell_type_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'donor_id', 'is_primary_data', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'annotation

In [6]:
#warnings.simplefilter("ignore", ResourceWarning)



model_dir = Path("/Users/doblado/Transformers-Architecture-in-Bioinformatics/Experimentacion/save/scGPT_human")



gene_col = "feature_name"
cell_type_key = "cell_type"

ref_embed_adata = scg.tasks.embed_data(
    adata_hvg,
    model_dir,
    gene_col=gene_col,
    obs_to_save=cell_type_key,  # optional arg, only for saving metainfo
    batch_size=64,
    use_fast_transformer=False,
    return_new_adata=True,
)

scGPT - INFO - match 2660/3000 genes in vocabulary of size 60697.


AttributeError: module 'os' has no attribute 'sched_getaffinity'

In [ ]:
ref_embed_adata.X.shape

: 

In [ ]:
sc.pp.neighbors(ref_embed_adata, use_rep="X")
sc.tl.umap(ref_embed_adata)

: 

In [ ]:
adata.obsm["X_scgpt"] = ref_embed_adata.X
adata.obsm["X_umap"] = ref_embed_adata.obsm["X_umap"]

: 

In [ ]:
# Add the current index ('ensembl_id') as a new column
adata.var['ensembl_id'] = adata.var.index

# Set the new index to the 'feature_name' column
adata.var.set_index('feature_name', inplace=True)

: 

In [ ]:
# Add a copy of the gene symbols back to the var dataframe
adata.var['gene_symbol'] = adata.var.index

: 

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    #sc.pp.neighbors(ref_embed_adata, use_rep="X")
    #sc.tl.umap(ref_embed_adata)
    sc.pl.umap(adata, color=["cell_type", "annotation_res0.34_new2"], wspace = 0.6)

: 

In [ ]:
sc.pl.umap(adata, color=['cell_type', 'MKI67', 'LYZ', 'RBP2', 'MUC2', 'CHGA', 'TAGLN', 'ELAVL3'], frameon=False, use_raw=False, legend_fontsize ="xx-small", legend_loc="none")

: 